In [1]:
import numpy as np

In [77]:
def nulldiagonalelem(m):
    for i in range(len(m[1,:])):
        if m[i,i] == 0:
            return True
    return False

def diagonaldominant(m):
    for i in range(len(m[1,:])):
        sum = 0.0
        for j in range(len(m[1,:])):
            if i != j:
                sum += abs(m[i,j])

        if abs(m[i,i]) <= sum:
            return False

    return True

def linecolcriterium(m):
    for i in range(len(m[1,:])):
        sum = 0.0
        for j in range(len(m[1,:])):
            if i != j:
                sum += abs(m[i,j])
            
        if sum >= 1:
            return False

    return True

def getLR(m):
    mnew = np.copy(m)
    for i in range(len(m[:,1])):
        mnew[i,:] = - mnew[i,:] / m[i,i]
        mnew[i,-1] = -mnew[i,-1]
        mnew[i,i] = 0

    return mnew

def getamp(m, b):
    return np.append(m, b[:,np.newaxis], axis=1)

def converges(m):
    if np.linalg.det(m) == 0:
        return False

    if nulldiagonalelem(m):
        return False

    mnew = getLR(m) 

    if diagonaldominant(m) or linecolcriterium(mnew) or linecolcriterium(mnew.T):
        return True  

    return False

def reltol(x1, x0):
    max = 0.0

    for i in range(len(x1)):
        if x0[i] == 0:
            if abs(x1[i] - x0[i]) > max:
                max = abs(x1[i] - x0[i])
                continue
        else:
            if abs(x1[i] - x0[i])/abs(x0[i]) > max:
                max = abs(x1[i] - x0[i])/abs(x0[i])  

    return max

def stepJacobiRichardson(LRb, x):
    LR = LRb[:,:-1]
    b = LRb[:,-1]

    return LR.dot(x) + b

def stepGaussSeidel(LRb, x):
    LR = LRb[:,:-1]
    b = LRb[:,-1]
    L = np.tril(LR)
    R = np.triu(LR)

    xnew = np.zeros(len(x))

    for i in range(len(x)):
        xnew[i] = L[i,:].dot(xnew) + R[i,:].dot(x) + b[i]

    return xnew

def solveJacobiRichardson(m, b, maxiter=100000, rtol=1e-4):
    mamp = getamp(m, b)
    LRb = getLR(mamp)
    x0 = LRb[:,-1]
    x2 = np.copy(x0)
    x1 = np.copy(x0)

    for i in range(maxiter):
        x2 = stepJacobiRichardson(LRb, x1)
        if reltol(x2, x1) < rtol:
            return x2
            
        x1 = np.copy(x2)

    return x2

def solveGaussSeidel(m, b, maxiter=100000, rtol=1e-4):
    mamp = getamp(m, b)
    LRb = getLR(mamp)
    x0 = LRb[:,-1]
    x2 = np.copy(x0)
    x1 = np.copy(x0)

    for i in range(maxiter):
        x2 = stepGaussSeidel(LRb, x1)
        if reltol(x2, x1) < rtol:
            return x2
            
        x1 = np.copy(x2)

    return x2

In [78]:
m = np.array([10.0, 2.0, 1.0, 1.0, 5.0, 1.0, 1.0, 3.0, 10.0]).reshape(3,3)
b = np.array([7.0, -8.0, 6.0])
sol = solveGaussSeidel(m, b, rtol=1e-4)
print(sol)

[ 0.99333331 -2.0200001   1.1066667 ]
